In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#1

In [3]:
def get_data(token):
    if token in ['ETH', 'SOL','USDT','FLOW']:
        server="kraken"
    else: 
        server="coinbase-pro"
    res = requests.get(
        f'https://api.cryptowat.ch/markets/{server}/{token}usd/ohlc',
        params={
            'periods': '3600',
            'after': str(int(pd.Timestamp('2021-11-22').timestamp()))
        }
    )
 
    df = pd.DataFrame(
        res.json()['result']['3600'],
        columns=['ts', 'open', 'high', 'low', 'close', 'volume', 'volumeUSD']
    )
    df['ts'] = pd.to_datetime(df.ts, unit='s')
    df['token'] = token
    
    return df

In [4]:
tokens = ['ETH', 'SOL', 'AVAX', 'USDT', 'FLOW']

df = pd.concat([
    get_data(token)
    for token in tokens
])

In [5]:
df

,ts,open,high,low,close,volume,volumeUSD,token
0,2021-11-22 00:00:00,4316.410,4339.250,4250.000,4263.510,2275.049242,9.768715e+06,ETH
1,2021-11-22 01:00:00,4265.390,4269.980,4210.000,4234.280,5177.368685,2.191977e+07,ETH
2,2021-11-22 02:00:00,4234.280,4244.140,4173.000,4217.000,2137.011937,8.986250e+06,ETH
3,2021-11-22 03:00:00,4218.090,4219.480,4163.640,4191.830,1377.294702,5.782201e+06,ETH
4,2021-11-22 04:00:00,4191.820,4211.210,4130.430,4166.580,1762.426504,7.340491e+06,ETH
...,...,...,...,...,...,...,...,...
649,2021-12-19 01:00:00,8.481,8.489,8.469,8.489,2180.562291,1.849647e+04,FLOW
650,2021-12-19 02:00:00,8.486,8.493,8.452,8.458,8585.777571,7.270974e+04,FLOW
651,2021-12-19 03:00:00,8.469,8.553,8.469,8.553,2287.398851,1.947771e+04,FLOW
652,2021-12-19 04:00:00,8.565,8.610,8.565,8.600,1797.236578,1.544555e+04,FLOW


In [6]:
#2

In [7]:
df.groupby("token")["volumeUSD"].sum().to_frame().sort_values(by="volumeUSD",ascending=False)

,volumeUSD
token,
ETH,4.799298e+09
AVAX,4.388504e+09
USDT,4.089213e+09
SOL,8.386712e+08
FLOW,9.655595e+07


In [8]:
#3

In [9]:
close_ratio = df[df['token']=='ETH']['close'] / df[df['token']=='SOL']['close']
close_ratio = close_ratio.rename('close price ratio between ETH and SOL')
df = df.merge(close_ratio, how='left', left_index=True, right_index=True)

In [10]:
df

,ts,open,high,low,close,volume,volumeUSD,token,close price ratio between ETH and SOL
0,2021-11-22 00:00:00,4316.4100,4339.2500,4250.0000,4263.5100,2.275049e+03,9.768715e+06,ETH,18.501606
0,2021-11-22 00:00:00,230.3000,234.5300,228.6200,230.4400,5.034093e+03,1.166822e+06,SOL,18.501606
0,2021-11-22 00:00:00,131.6900,133.7900,127.7000,128.6700,9.686748e+04,1.267080e+07,AVAX,18.501606
0,2021-11-22 00:00:00,1.0009,1.0010,1.0008,1.0009,4.405672e+06,4.409661e+06,USDT,18.501606
0,2021-11-22 00:00:00,13.8870,14.1690,13.8870,14.1500,2.326138e+04,3.274755e+05,FLOW,18.501606
...,...,...,...,...,...,...,...,...,...
653,2021-12-19 05:00:00,3991.0300,4009.9900,3991.0300,4001.6000,2.607529e+02,1.043628e+06,ETH,21.667750
653,2021-12-19 05:00:00,184.7900,185.1000,184.4000,184.6800,2.630396e+03,4.860296e+05,SOL,21.667750
653,2021-12-19 05:00:00,115.1000,115.3600,114.6700,114.8300,9.156996e+03,1.053430e+06,AVAX,21.667750
653,2021-12-19 05:00:00,1.0005,1.0005,1.0004,1.0004,1.413561e+05,1.414199e+05,USDT,21.667750


In [11]:
#4

In [12]:
df = df.rename(columns={'volume': 'volumeBase', 'volumeUSD': 'volumeTerm'})

In [13]:
df

,ts,open,high,low,close,volumeBase,volumeTerm,token,close price ratio between ETH and SOL
0,2021-11-22 00:00:00,4316.4100,4339.2500,4250.0000,4263.5100,2.275049e+03,9.768715e+06,ETH,18.501606
0,2021-11-22 00:00:00,230.3000,234.5300,228.6200,230.4400,5.034093e+03,1.166822e+06,SOL,18.501606
0,2021-11-22 00:00:00,131.6900,133.7900,127.7000,128.6700,9.686748e+04,1.267080e+07,AVAX,18.501606
0,2021-11-22 00:00:00,1.0009,1.0010,1.0008,1.0009,4.405672e+06,4.409661e+06,USDT,18.501606
0,2021-11-22 00:00:00,13.8870,14.1690,13.8870,14.1500,2.326138e+04,3.274755e+05,FLOW,18.501606
...,...,...,...,...,...,...,...,...,...
653,2021-12-19 05:00:00,3991.0300,4009.9900,3991.0300,4001.6000,2.607529e+02,1.043628e+06,ETH,21.667750
653,2021-12-19 05:00:00,184.7900,185.1000,184.4000,184.6800,2.630396e+03,4.860296e+05,SOL,21.667750
653,2021-12-19 05:00:00,115.1000,115.3600,114.6700,114.8300,9.156996e+03,1.053430e+06,AVAX,21.667750
653,2021-12-19 05:00:00,1.0005,1.0005,1.0004,1.0004,1.413561e+05,1.414199e+05,USDT,21.667750


In [14]:
#5

In [21]:
token_close_price = df.pivot_table(values='close', index='ts', columns='token')

In [29]:
token_close_price

token,AVAX,ETH,FLOW,SOL,USDT
ts,,,,,
2021-11-22 00:00:00,128.67,4263.51,14.150,230.44,1.0009
2021-11-22 01:00:00,134.85,4234.28,13.621,226.09,1.0010
2021-11-22 02:00:00,132.54,4217.00,13.753,227.15,1.0009
2021-11-22 03:00:00,133.38,4191.83,13.652,223.14,1.0009
2021-11-22 04:00:00,136.29,4166.58,13.581,220.73,1.0009
...,...,...,...,...,...
2021-12-19 01:00:00,114.36,3944.51,8.489,182.72,1.0004
2021-12-19 02:00:00,114.08,3907.02,8.458,180.54,1.0004
2021-12-19 03:00:00,114.76,3944.00,8.553,181.59,1.0005


In [23]:
#6

In [ ]:
log_return = np.log(token_close_price / token_close_price())

In [ ]:
log_return